In [4]:
# %%capture
# !pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
# !pip install llama-cpp-python==0.2.69

## Get Phi-3 Mini
Fetch the compressed Phi-3 model with half precision of FP16 compared to the original model with full precision of FP32.

In [2]:
# !wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

Der Befehl "wget" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


## Load Model

In [6]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="models/Phi-3-mini-4k-instruct-fp16.gguf", # Path to model
    n_gpu_layers=-1,  # All the layers of the model run on the GPU
    max_tokens=500,   # limits the output length (how much text the model will generate in response).
    n_ctx=2048,       # determines the input capacity (how much context the model can handle).
    seed=42,
    verbose=False
)

## Basic Chain
Combines Prompt with LLM Model

In [31]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>{input_prompt}<|end|><|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

# Connect the prompt module with the model
basic_chain = prompt | llm

# Input the prompt
basic_chain.invoke(
    {
    "input_prompt": "How many states are there in Germany?"
    }
)

c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Germany is a federal republic comprising 16 states, known as "Bundesländer" in German. These states are Baden-Württemberg, Bavaria (Bayern), Berlin, Brandenburg, Bremen, Hamburg, Hesse (Hessen), Mecklenburg-Vorpommern, North Rhine-Westphalia (Nordrhein-Westfalen), Saarland, Saxony (Sachsen), Saxony-Anhalt (Sachsen-Anhalt), Schleswig-Holstein, Thuringia (Thüringen), Lower Saxony (Niedersachsen), and Upper Saxony (Oberlausitz).'

## Multiple Chains
Combines multiple modules together

In [10]:
from langchain import LLMChain

product_template = """<s><|user|>Suggest one product in 2-5 words which could be advertised based on a short description: {idea}. Only return the product which could be advertised.<|end|><|assistant|>"""
product_prompt = PromptTemplate(
    template=product_template,
    input_variables=["idea"]
)
product_description = LLMChain(llm=llm, prompt=product_prompt, output_key="product_description")
product_description.invoke(
    {"idea": "An old lady is knitting a pullover while humming a tune"}
)

C:\Users\sagarwa2\AppData\Local\Temp\ipykernel_12392\1635870151.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  product_description = LLMChain(llm=llm, prompt=product_prompt, output_key="product_description")
c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'idea': 'An old lady is knitting a pullover while humming a tune',
 'product_description': ' Cozy knitted pullover yarn.'}

In [11]:
product_name_slogan_template = """<s><|user|>Suggest one name and one slogan for the product based on the idea: {idea} and product: {product_description}<|end|><|assistant|>"""
product_name_slogan_prompt= PromptTemplate(
    template=product_name_slogan_template,
    input_variables=["idea", "product_description"]
)
product_name_slogan = LLMChain(llm=llm, prompt=product_name_slogan_prompt, output_key="product_name_slogan")

In [12]:
product_ad_template = """<s><|user|>Give a really short advertisement idea to pitch to the team in 4-5 sentences based on the idea: {idea}, product description: {product_description}. Also, consider the name and slogan of the product, which is: {product_name_slogan}<|end|><|assistant|>"""
product_ad_prompt = PromptTemplate(
    template=product_ad_template,
    input_variables=["idea", "product_name_slogan", "product_description"]
)

product_ad = LLMChain(llm=llm, prompt=product_ad_prompt, output_key="product_ad")

In [13]:
product_chain = product_description | product_name_slogan | product_ad
product_chain.invoke("An old lady is knitting a pullover while humming a tune")

c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'idea': 'An old lady is knitting a pullover while humming a tune',
 'product_description': ' Cozy knitted pullover!',
 'product_name_slogan': ' Name: "Elderly Elegance Wrap"\n\nSlogan: "Warmth in Every Stitch - Embracing Comfort and Style, One Thread at a Time."',
 'product_ad': ' Introducing "Elderly Elegance Wrap" - a cozy knitted pullover that brings comfort and style to your loved ones. Picture an elderly lady, serenely knitting her signature piece with a smile on her face as she hums a familiar tune. Our unique product is not just another pullover; it\'s warmth in every stitch - embracing comfort and style one thread at a time! Experience the essence of Elderly Elegance Wrap, where timeless craftsmanship meets modern elegance. Don\'t miss out on this must-have for your loved ones\' cozy winter collection!'}

## Conversation Memory

In [27]:
from langchain import PromptTemplate
from langchain.memory import ConversationSummaryMemory

summary_template="""<s><|user|>Summarize the conversation and update with new lines.
Current summary: {summary}
New lines: {new_lines}
Updated summary: <|end|><|assistant|>
"""
summary_prompt=PromptTemplate(
    template=summary_template,
    input_variables=["summary", "new_lines"]
)

memory = ConversationSummaryMemory(llm=llm, prompt=summary_prompt, memory_key="chat_history")

In [28]:
# Create a simple prompt
template = """<s><|user|>Current conversation: {chat_history}
{input_prompt}<|end|><|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

# Link LLM chain to conversation memory
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

llm_chain.invoke({"input_prompt": "Hey, I am visiting Germany in September. Do you know any breweries to try some nice beer?"})

c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hey, I am visiting Germany in September. Do you know any breweries to try some nice beer?',
 'chat_history': '',
 'text': ' Absolutely! Germany is known as the beer paradise, and September offers fantastic opportunities to explore various breweries. Here are a few must-visit ones:\n\n1. Augustiner Brewery in Munich - Located near Marienplatz, this traditional Bavarian brewery has been producing beer since 1328. Try their renowned Augustiner Helles lager and enjoy the festive atmosphere at Oktoberfest (although it takes place earlier in September).\n\n2. Hofbrauhaus Munich - Another iconic Munich-based brewery with a rich history dating back to 1589. They offer tours, traditional Bavarian food, and have several beer gardens where you can taste their famous Märzen lager.\n\n3. Weihenstephan Brewery in Freising - As one of the oldest breweries in the world (founded in 768), they\'re also located within walking distance from Munich, making it convenient to visit both plac

In [30]:
llm_chain.invoke(
    {"input_prompt": "Which month am I traveling to Germany?"})

c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
c:\Users\sagarwa2\AppData\Local\miniconda3\envs\thellmbook\lib\site-packages\llama_cpp\llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Which month am I traveling to Germany?',
 'chat_history': ' While visiting Germany in September, you\'re interested in exploring various breweries to enjoy some nice beer. Here are seven must-visit brewery destinations:\n\n1. Augustiner Brewery in Munich offers traditional Bavarian experiences with its Augustiner Helles lager and Oktoberfest tours.\n2. Hofbrauhaus Munich, dating back to 1589, provides tours, food, and beer gardens for their Märzen lager.\n3. Weihenstephan Brewery in Freising is one of the world\'s oldest breweries (founded in 768), offering diverse tastings near Munich.\n4. Bräustüberl Weinstube Göttersburg in Bamberg serves local specialties like Franconian Altbier and Heller Bock, with regional sausages on the menu.\n5. Paulaner Brauerei in Munich has an engaging tour where visitors can learn about brewing and taste a wide variety of beers.\n6. Spaten Brewery Am Glockenspiel showcases its long history by providing tours, beer garden experiences, and